# Tests on YoloV8 for fAIr
Using model V2 from Omdena results.
Mocking `test_yolo_v2.py`

In [2]:
# Standard library imports
import os
import time
import warnings
import ultralytics

import csv

os.environ.update(os.environ)
os.environ["RAMP_HOME"] = os.getcwd()

In [3]:
# Reader imports
from hot_fair_utilities import polygonize, predict, preprocess
from hot_fair_utilities.preprocessing.yolo_v8_v2.yolo_format_anna import yolo_format
from hot_fair_utilities.training.yolo_v8_v2.train import train as train_yolo

warnings.simplefilter(action="ignore", category=FutureWarning)


Ramp eval metrics are not available, Possibly ramp is not installed


In [ ]:
# Define basic variables
# base_path = f"{os.getcwd()}/ramp-data/sample_2"
base_path = '/Users/azanchetta/fAIr_metric'
data_path = f'{base_path}/training_results'
preprocessed_data_path=f'{base_path}/metric_data'
#  Obtain cities list folders name in data folder
# cities_list = ['modelfake', 'model149_td489'] # sample of names, for tests
cities_list= [ item for item in os.listdir(preprocessed_data_path) if os.path.isdir(os.path.join(preprocessed_data_path, item)) ]

NameError: name 'root' is not defined

In [ ]:
for i in cities_list:
    print(i)

In [ ]:
# Looping through the cities list, and check per each folder that tiles number is consistent, and the shapes too
for city in cities_list:
    # city_folder_name = f'{data_path}/{city}/train'
    city_folder_name=f'{base_path}/metric_data/{city}'
    csv_file_basepath = f'{data_path}/{city}/train'
    print(f'\n---\nNow working on {city}\n---')
    print(f'city folder name is {city_folder_name}\n\n---\n')


    class print_time:
        def __init__(self, name):
            self.name = name

        def __enter__(self):
            self.start = time.perf_counter()
            return self

        def __exit__(self, type, value, traceback):
            print(f"{self.name} took {round(time.perf_counter() - self.start, 2)} seconds")

    start_time = time.time()

### SKIP fAIr processing, we already have the images ready

    # city_data_dir = f'{base_path}/training_results/{city}/train'
    city_data_dir = f'{city_folder_name}'
    yolo_data_dir = f'{base_path}/yolo_v2'
    print(f'city is {city}')
    with print_time("yolo conversion"):
        print(f'\n___ Starting yolo files conversion\n')
        yolo_format(
            input_path=city_data_dir,
            csv_path=csv_file_basepath,
            output_path=yolo_data_dir,
            city_name=city
        )


In [ ]:
# Training

output_model_path,output_model_iou_accuracy = train_yolo(
    data=f"{base_path}",
    weights=f"{os.getcwd()}/yolov8s_v2-seg.pt", 
    # gpu="cpu",
    epochs=2,
    batch_size=16,
    pc=2.0,
    output_path=yolo_data_dir,
    dataset_yaml_path=os.path.join(yolo_data_dir,'yolo_dataset.yaml')
)
print(output_model_iou_accuracy)


In [ ]:
# Prediction
prediction_output = f"{base_path}/prediction/output"
# model_path = f"{output_path}/weights/best.pt"
with print_time("inference"):
    predict(
        checkpoint_path=output_model_path,
        input_path=f"{base_path}/prediction/input",
        prediction_path=prediction_output,
    )

In [ ]:
# # Deal with the csv files with list of train/val/pred images used in RAMP
# #  testing function ... this goes inside `fined_files` in yolo_format_anna.py
# city="model51_td364"
# city_folder_name=f'{data_path}/{city}/train'
# csv_file_name = f'fair_split_train.csv'
# csv_file_path = f'{city_folder_name}/{csv_file_name}'
# print(f'CSV file is {csv_file_name}')
# print(f'CSV file is {csv_file_path}')
# csv_raw_list = []

# with open(csv_file_path, "r") as file_obj:
#     heading = next(file_obj)
#     reader_obj = csv.reader(file_obj, delimiter="\t")
#     for row in reader_obj:
#         csv_raw_list.append(row)
# print(f'this is the list from the csv file:\n{csv_raw_list}')
# csv_nested_list = []
# for ccc in csv_raw_list:
#     # print(ccc)
#     nested = ccc[0]
#     # print(f'nested {nested}')
#     name_csv = nested.split('/')[-1]
#     csv_nested_list.append(name_csv)

# # filenames_from_csv = [csvi.split("/",1)[-1] for csvi in csv_nested_list] # this is to get the last element of the string (i.e. the file name)
# # print(f'filenames hopefully {filenames_from_csv}')
# print(f'is this the names? {csv_nested_list}')